# Internal control experiments

This notebooks runs some experiments with internal bandit NRM control. 

For the application of the NRM model to resource management to one
computational job, the global resource optimization problem is the following:

$$
\begin{array}{l}
    \min \quad e_{\text{total}} \\
	\text{s.t.} \quad  t > \tau t_{\text{ref}}
\end{array}
$$

Where $e_{\text{total}}$ denotes the total energy spent by the system during
the lifetime of the job, whose duration is denoted by $t^T$. We denote by
$t_{\text{ref}}$ a reference measurement of the runtime of the job on an
unmanaged system. $\tau <1$ is a parameter controlling the amount of runtime
degradation allowed for the job.

The value of this global objective can be easily measured a-posteriori for a
computational job using power instrumentation techniques. Assuming both
workload and platform behavior to be deterministic, this objective is measured
using two runs of the system: A first run without resource management to
acquire $t_{\text{ref}}$, and one run with NRM enabled. In order for NRM's
round-based control strategy to address this problem, we need an online loss
value however. This loss is obtained using the following loose assumptions:

- The passive power consumption of the node is fixed and known. [1]

- The total power consumption in a given time period can be estimated as
  the sum of the static node consumption over that period and the RAPL power
  measurement over that period. [2]

- The impact of a choice of power-cap on the job's runtime can be
  interpolated linearly from its impact on CPU counters. [3]


Denoting as in the previous section the round counter by $0<r<T$, the known
passive static power consumption by $p_{\text{static}}$, the starting time of
the job by $t^0$ and the end time of round $r$ by $t^r$, we can write the total
energy expenditure of the job based on RAPL power measurements $p^r$ using
assumptions 1 and 2 as:

$$
	e_{\text{total}} = \sum_{r=1}^{r=T} (p^r + p_{\text{static}}) (t^{r-1} - t^{r})
$$

Using assumption 3 means that we can reasonably estimate the
change in job runtime incurred by the choice of power-cap in round $r$ by
evaluating $\frac{s^r_{\text{ref}}}{s^r}$. We use this as part of our proxy
cost in two ways. First, this quantity is used to evaluate breaching of the
constraint on $t$, and second, it is used to adjust for an expected increase in
the number of rounds due to the impact on job runtime. This gives rise to the
following value for the loss at round $r$:

$$
	\ell^r = \mathbb{\huge 1}_{\left( \frac{s^r}{s^r_{\text{ref}}}>\tau \right)}
   \left( \frac{s^r_{\text{ref}}}{s^r} \left( p^r + p_{\text{static}} \right) \right)
$$


In [1]:
cd ..

/home/fre/workspace/hnrm


In [2]:
%%capture
%%bash
./shake.sh build # for the daemon 
./shake.sh client # for the upstream client
./shake.sh pyclient # for the shared client libraryt

In [2]:
%%bash
dhall-to-json <<< '
    let 
        Cfg = ./resources/types/Cfg.dhall
    in 
        (./resources/defaults/Cfg.dhall // {controlCfg = Some {
         minimumControlInterval = {fromuS = 1000000.0},
         learnCfg = < Lagrange :{lagrangeConstraint: Double}|
                      Knapsack :{knapsackConstraint: Double} >.Lagrange {
                        lagrangeConstraint = 1.0 
                      },
          speedThreshold = 0.9,
        referenceMeasurementRoundInterval = +10
        }}):Cfg'

{"pmpi_lib":"pmpi_lib","verbose":"Error","logfile":"/tmp/nrm.log","singularity":false,"argo_nodeos_config":"argo_nodeos_config","controlCfg":{"referenceMeasurementRoundInterval":10,"learnCfg":{"lagrangeConstraint":1},"speedThreshold":0.9,"minimumControlInterval":{"fromuS":1000000}},"upstreamCfg":{"upstreamBindAddress":"*","rpcPort":3456,"pubPort":2345},"libnrmPath":null,"perf":"perf","argo_perf_wrapper":"nrm-perfwrapper","downstreamCfg":{"downstreamBindAddress":"ipc:///tmp/nrm-downstream-event"},"nodeos":false,"hwloc":"hwloc","raplCfg":{"raplFrequency":{"fromHz":1},"raplPath":"/sys/devices/virtual/powercap/intel-rapl"},"dummy":true,"slice_runtime":"Dummy","hwmonCfg":{"hwmonPath":"/sys/class/hwmon","hwmonEnabled":true}}


In [3]:
%load_ext nb_black
import json

daemonCfg = {
    "controlCfg": {
        "referenceMeasurementRoundInterval": 10,
        "learnCfg": {"lagrangeConstraint": 1},
        "speedThreshold": 0.9,
        "minimumControlInterval": {"fromuS": 1000000},
    },
    "verbose": "Debug",
}


print(daemonCfg)

workloads = {
    "dummy": [
        {
            "cmd": "stream_c",
            "args": [],  # running the sleep command for 10 seconds.
            "sliceID": "toto",
            "manifest": {
                "app": {
                    "slice": {"cpus": 1, "mems": 1},
                    "perfwrapper": {
                        "perfLimit": {"fromOps": 100000},
                        "perfFreq": {"fromHz": 1},
                    },
                },
                "name": "perfwrap",
            },
        }
    ]
}

{'controlCfg': {'referenceMeasurementRoundInterval': 10, 'learnCfg': {'lagrangeConstraint': 1}, 'speedThreshold': 0.9, 'minimumControlInterval': {'fromuS': 1000000}}, 'verbose': 'Debug'}


<IPython.core.display.Javascript object>

In [4]:
import nrm.tooling as nrm

host = nrm.Local()

<IPython.core.display.Javascript object>

In [5]:
host.start_daemon(daemonCfg)
assert host.check_daemon()

cpd = host.get_cpd()
print("Pretty-printing the control problem description:\n %s" % cpd)
cpdd = dict(cpd)
print("Python dictionary version: %s" % cpdd)

connecting
connected to tcp://localhost:2345
Pretty-printing the control problem description:
 Problem 
    { sensors = Map 
        [ 
            ( SensorID { sensorID = "RaplKey (PackageID 0)" }
            , Sensor 
                { range = 0.0 ... 2.6214332885e11
                , maxFrequency = 3.0
                } 
            ) 
        ]
    , actuators = Map 
        [ 
            ( ActuatorID { actuatorID = "RaplKey (PackageID 0)" }
            , Actuator 
                { actions = 
                    [ DiscreteDouble 1.8e8
                    , DiscreteDouble 2.0e8
                    ] 
                }
            ) 
        ]
    , objectives = 
        [ 
            ( 1.0
            , OValue 
                ( SensorID { sensorID = "RaplKey (PackageID 0)" } )
            ) 
        ]
    , constraints = []
    } 
Python dictionary version: {'objectives': [[1, {'oValue': 'RaplKey (PackageID 0)'}]], 'constraints': [], 'sensors': [['RaplKey (PackageID 0)', {'maxFr

<IPython.core.display.Javascript object>

In [6]:
s = host.get_state()
print(s)
print(dict(s))

NRMState 
    { pus = Map 
        [ 
            ( PUID 0
            , PU
            ) 
        , 
            ( PUID 4
            , PU
            ) 
        , 
            ( PUID 1
            , PU
            ) 
        , 
            ( PUID 5
            , PU
            ) 
        , 
            ( PUID 2
            , PU
            ) 
        , 
            ( PUID 6
            , PU
            ) 
        , 
            ( PUID 3
            , PU
            ) 
        , 
            ( PUID 7
            , PU
            ) 
        ] 
    , cores = Map 
        [ 
            ( CoreID 0
            , Core
            ) 
        , 
            ( CoreID 1
            , Core
            ) 
        , 
            ( CoreID 2
            , Core
            ) 
        , 
            ( CoreID 3
            , Core
            ) 
        ] 
    , packages = Map 
        [ 
            ( PackageID 0
            , Package 
                { rapl = Just 
                    ( Rapl 
       

<IPython.core.display.Javascript object>

The next cell just stops the daemon cleanly.

In [7]:
host.stop_daemon()
assert host.check_daemon() == False

<IPython.core.display.Javascript object>

### Dummy experiment: printing measurements

We now are ready to run an external resource management strategy. The next cell uses the low-level "upstream" message passing interface to run a dummy strategy that takes a constant action. Message schemas for this API can be browsed in the [./resources/](./resources/) folder.

In [ ]:
import time
from collections import defaultdict

for name, workload in workloads.items():
    host.start_daemon(daemonCfg)
    print("Starting the workload")
    host.run_workload(workload)
    history = defaultdict(list)
    print(host.get_state())
    getCPD = True
    while host.check_daemon() and not host.workload_finished():
        measurement_message = host.workload_recv()
        msg = json.loads(measurement_message)
        # print(msg)
        if "pubMeasurements" in msg:
            if getCPD:
                getCPD = False
                time.sleep(1)
                cpd = dict(host.get_cpd())
                print("Sensor identifier list:")
                for sensorID in [sensor[0] for sensor in cpd["sensors"]]:
                    print("- %s" % sensorID)
                print("Actuator identifier list:")
                for sensorID in [sensor[0] for sensor in cpd["actuators"]]:
                    print("- %s" % sensorID)
            content = msg["pubMeasurements"][1][0]
            t = content["time"]
            sensorID = content["sensorID"]
            x = content["sensorValue"]
            print(
                "Measurement: originating at time %s for sensor %s of value %s"
                % (content["time"], content["sensorID"], content["sensorValue"])
            )
            history[sensorID].append((t, x))

        if "pubAction" in msg:
            print(msg)
            content = msg["pubAction"][1]
            t = msg["pubAction"][0]
            actuatorID = content["actuatorID"] + "(action)"
            x = content["actuatorValue"]
            controller=msg["pubAction"][2]
            print(controller)
            history[actuatorID].append((t, x))
            if "bandit" in controller.keys():
                print("DEBUG")
                for arm in controller["bandit"]["lagrange"]["lagrangeConstraint"]["weights"]:
                    value =arm["action"][0]["actuatorValue"]
                    history[str(value)+"-probability"].append((t,arm["probability"]["getProbability"]))
                    history[str(value)+"-cumulativeLoss"].append((t,arm["cumulativeLoss"]["getCumulativeLoss"]))
            print(
                "Action: originating at time %s for actuator %s of value %s"
                % (t,actuatorID,x)
            )

                

    host.check_daemon()
host.stop_daemon()


connecting
connected to tcp://localhost:2345
Starting the workload
NRMState 
    { pus = Map 
        [ 
            ( PUID 0
            , PU
            ) 
        , 
            ( PUID 4
            , PU
            ) 
        , 
            ( PUID 1
            , PU
            ) 
        , 
            ( PUID 5
            , PU
            ) 
        , 
            ( PUID 2
            , PU
            ) 
        , 
            ( PUID 6
            , PU
            ) 
        , 
            ( PUID 3
            , PU
            ) 
        , 
            ( PUID 7
            , PU
            ) 
        ] 
    , cores = Map 
        [ 
            ( CoreID 0
            , Core
            ) 
        , 
            ( CoreID 1
            , Core
            ) 
        , 
            ( CoreID 2
            , Core
            ) 
        , 
            ( CoreID 3
            , Core
            ) 
        ] 
    , packages = Map 
        [ 
            ( PackageID 0
            , Package 

In [ ]:
import pandas as pd

for sensorID, measurements in history.items():
    dataframe = pd.DataFrame(
        data=[(pd.Timestamp(t, unit="us"), m) for t, m in measurements]
    )
    dataframe.columns = ["time", "value"]
    dataframe = dataframe.set_index("time")
    ax = dataframe.plot(
        marker="o", linestyle="-", title='values for sensor "%s[..]"' % sensorID[:20]tth
    )

In [ ]:
print(history)